# Rich Club Algorithm

## Data Preparation

Packages

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Read data and bring into a fromat that can be transformed into a graph

In [ ]:
with open("data/followers_complete.csv") as file:
    lines = list(file)
    lines = [line.rstrip('\n') for line in lines]
    data = [tuple(line.split(",")) for line in lines]

Check

In [ ]:
data[:5]

Create empty graph `G`

In [ ]:
G = nx.Graph()

Add nodes from `data` list of tuples and check if number of nodes is correct

In [ ]:
G.add_nodes_from(d[0] for d in data)
G.number_of_nodes()

Add edges from `data` list of tuples and check number of edges

In [ ]:
G.add_edges_from(data)
G.number_of_edges()

## Theory

Highly connected nodes are "hubs", but how are these highly connected hubs connected to **each other**? $\rightarrow$ the node is important, how is it connected to other important nodes?

Answer: **rich club properties** of a graph $\rightarrow$ richness of a specific node.

* Start with an unweighted Graph `G`
* The **richest nodes** are defined as the densely connected nodes in the graph which points to an **interlinked core of high degree nodes**. These cores are hubs in the network, but at the same time extremely highly interconnected.

Definition of the **rich club coefficient**:
We can define the rich club coefficient $\Phi(k)$ of a Graph $G$ at a level $k$ or at a degree $k$ as the **proportion of the number of edges at nodes of degree** $>k$ (strictly larger than $k$) divided by the total number of edges in the undirected subgraph:

$$
\Phi(k) = \frac{2E_{>k}}{N_{>k}(N_{>k}-1)}
$$

If you look a subgraph and you want to compute the rich club coefficient of this subgraph, you look at the number of edges at nodes of degree $>k$ (you count the number of edges that are connected to edges with degrees $>k$) and divide them by the total number of total edges in this subgraph.

* If $\Phi(k) = 0$, there are no edges of nodes with degree $>k$. $k$ here is a *cut-off value of richness* where richness is defined in terms of degree. 
* $\Phi(k)$ (richnessness in terms of degree) quantifies the connectivity between the richest nodes in a graph for a given level of $k$ of that graph. It tells us how rich the nodes in that subgraph are.

When using the rich club algorithm implementation in `NetworkX`, it returns a **dictionary, keyed by degree, with rich-club coefficient values**.

In graph theory, the **degree** is the number of edges that are **incident** to the node. A node is **incident** to an edge if the node is one of the two nodes connected by the edge. The **maximum degree** of a graph $G$, denoted by $\Delta(G)$, and the **minimum degree** of a graph, denoted by $\delta(G)$, are the maximum and minimum degree of its nodes. In the graph below, the nodes are labelled by degree.

* $\Delta(G) = 5$ (in a multigraph, loops are counted twice).
* $\delta(G) = 0$

<img src="data/degrees.png"></img>

In more simple terms, the degree of a graph node $v$ of a graph $G$ is **the number of graph edges which touch** $v$.


**Sources**:

* https://www.youtube.com/watch?v=Z_O-q10MXuY
* https://en.wikipedia.org/wiki/Rich-club_coefficient#:~:text=The%20rich%2Dclub%20coefficient%20is,between%20nodes%20of%20high%20degree.
* https://de.wikipedia.org/wiki/Teilgraph
* https://en.wikipedia.org/wiki/Degree_(graph_theory)
* https://mathworld.wolfram.com/VertexDegree.html
* https://en.wikipedia.org/wiki/Incidence_(graph)
* https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.richclub.rich_club_coefficient.html#networkx.algorithms.richclub.rich_club_coefficient

## Practice

Now we check the rich club coefficients:

In [ ]:
rc = nx.rich_club_coefficient(G, normalized = False, seed = 42)
rc

According to the `NetworkX` documentation, this dictionary shows the rich-club coefficient values, **keyed by degree**. Therefore, we check the degrees of the nodes in the network:

In [ ]:
degrees = list(G.degree)
sorted(degrees, key = lambda x: x[1], reverse = True)

Let's create a few plots. First the `rcc` dict, i.e. the rich club coefficients against the degrees:

In [ ]:
lists = sorted(rc.items())
x, y = zip(*lists)

plt.figure(figsize=(22,16))
plt.plot(x, y, label = "Rich Club Coefficient")
plt.title(r"Evolution of the Rich Club Coefficient", fontsize = 22)
plt.xlabel("Node Degree", fontsize = 15)
plt.ylabel("$\Phi(k)$", fontsize = 15)
plt.legend(loc="upper left", fontsize = 15)
plt.grid(True, linestyle = ":")

# plt.savefig("img/RCC_Trump_Network.png")

The Rich Club Coefficient seems very high, even for nodes with Node Degree 0. Supposedly, this is because all the nodes in the network are connected somehow and there is no single node with node degree 0. Let's plot a histogram of the friends of all the accounts:

In [ ]:
# Prepare data (convert to numpy array):
friends = []
for d in degrees:
    friends.append(d[1])
friends_array = np.array(friends)

In [ ]:
### Create Plot:
plt.figure(figsize=(22,16))
plt.hist(friends_array, rwidth = 0.95, bins = len(np.unique(friends_array)))
plt.title("Distribution of Node Degrees", fontsize = 22)
plt.xlabel("Node Degree", fontsize = 15)
plt.ylabel("Number of Accounts", fontsize = 15)
plt.minorticks_on()
plt.grid(True, which = "major", linestyle = "-")
# plt.grid(True, which = "minor", linestyle = ":")

# plt.savefig("img/Node_Degrees_vs_Accounts.png")

Let's further inspect a few elements of the list of degrees: 

* The node `TomFitton`, according to the node degree, this node should have the most edges (95 edges).
* The node `NHC_Atlantic` should have the fewest edges with 7
* The node `realDonaldTrump` should have 84 edges.

In [ ]:
len(list(G.edges("TomFitton"))) == 95

In [ ]:
len(list(G.edges("NHC_Atlantic"))) == 7

In [ ]:
len(list(G.edges("realDonaldTrump"))) == 84

All of these hold true. Now, we assign the rich club coefficients to the corresponding nodes. The following function creates a dictionary with the account name and the corresponding RC values based on the node degree of each account.

In [ ]:
def rc_to_account(degrees):
    new = {}

    for i in degrees:
        element = {i[0]: i[1]}
        new.update(element)
    
    for key, value in new.items():
        deg = value
        for degree, rc_coef in rc.items():
            if deg == degree:
                new[key] = rc_coef
            elif deg >= 94:
                new[key] = 1.0
        
    return new

In [ ]:
rc_accounts = rc_to_account(degrees)
# rc_accounts

Let's plot a histogram like the one for Node Degrees, but for the RCC coefficients.

First, transform the `rc_accounts` dict into a list of tuples which can then be transformed to a Numpy array:

In [ ]:
# Create List of Tupels
rcc_list_of_tupels = []

for key, value in rc_accounts.items():
    rcc_list_of_tupels.append((key, value))

# Create Numpy Array
rcc_hist = []

for r in rcc_list_of_tupels:
    rcc_hist.append(r[1])
rcc_hist_array = np.array(rcc_hist)

In [ ]:
rcc_hist_array

In [ ]:
### Create Plot:
plt.figure(figsize=(22,16))
plt.hist(rcc_hist_array, rwidth = 0.95, bins = len(np.unique(rcc_hist_array)))
plt.title(r"Distribution of $\Phi(k)$", fontsize = 22)
plt.xlabel("Rich Club Coefficient", fontsize = 15)
plt.ylabel("Number of Accounts", fontsize = 15)
plt.grid(True, which = "major", linestyle = "-")

# plt.savefig("img/RCC_Distribution.png")

## Plots

### Plots of the entire network

Now let's try to plot the network according to the RC coefficients.

First, draw the Network from the first graph `G`. First, the RC coefficient is stored in a list which can be passed to the `draw` function. We'll also try to layer in order to highlight random nodes which will be later highlighted according to their RC coefficient.

In [ ]:
sizes = list(rc_accounts.values())

# For reproducibility of the plot:
np.random.seed(16)

# Define layout of the plot:
pos = nx.spring_layout(G)

# Highlight three randomly selected nodes as a test:
plt.figure(figsize=(20,14))
nx.draw(G, pos = pos, node_size = sizes, edge_color = "lightgray", width = 0.5, with_labels = True)
nx.draw(G.subgraph(["realDonaldTrump", "JennaEllisEsq", "ChuckGrassley"]), with_labels = True, pos = pos, node_color = "darkorange")

Now we will plot all Accounts where $\Phi(k) = 1.0$. First, we generate a list from the `rc_accounts` dict which only contains these items. This list will be used as a subgraph to hightlight these accounts within the network.

In [ ]:
rc_1 = []

for key, value in rc_accounts.items():
    item = []
    if value == 1.0:
        rc_1.append(str(key))
        
print(rc_1)

The following is a plot of the entire network with all the accounts where $\Phi(k) = 1.0$ and their connections to each other are highlighted.

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

plt.figure(figsize=(22,16))
nx.draw(G, 
        pos = pos, 
        node_size = 10,
        node_color = "mediumslateblue",
        edge_color = "lightgray",
        width = 0.5)
nx.draw(G.subgraph(rc_1), 
        pos = pos, 
        node_size = 60, 
        node_color = "royalblue",
        edge_color = "dimgray",
        width = 0.5,
        with_labels = True)

plt.savefig("img/Subnet_RCC_1.png")

Now it would be interesting to see how a few of these accounts are connected to the entire network. Once again, we create a graph which will be used as a subgraph to layer the individual network of one account. First, we start with `realDonaldTrump`. The data is taken straight from the `followers_complete` CSV file.

There are two options:

**1. Complete network of Donald Trump's friends with all edges**

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

plt.figure(figsize=(20,14))
nx.draw(G, 
        pos = pos, 
        node_size = 10,
        node_color = "royalblue",
        edge_color = "lightgray",
        width = 0.5)
nx.draw(G.subgraph(trump_graph), 
        pos = pos, 
        node_size = 60, 
        node_color = "mediumblue",
        edge_color = "dimgray",
        width = 0.5,
        with_labels = True)
nx.draw(G.subgraph("realDonaldTrump"), 
        pos = pos, 
        node_size = 500, 
        node_color = "navy",
        edge_color = "dimgray",
        width = 0.5,
        with_labels = True)

**2. Network of Donald Trump's friends which higlights only the edges between** `realDonaldTrump` **and the friends**:

In [ ]:
trump_graph = ["realDonaldTrump", "LouDobbs", "GOPLeader", "senatemajldr", "Jim_Jordan", "MariaBartiromo", "VP", 
               "GOPChairwoman", "parscale", "PressSec", "JesseBWatters", "WhiteHouse", "Scavino45", "KellyannePolls", 
               "IngrahamAngle", "Mike_Pence", "TeamTrump", "seanhannity", "CLewandowski_", "KatrinaPierson", 
               "foxandfriends", "GeraldoRivera", "ericbolling", "DanScavino", "EricTrump", "DonaldJTrumpJr", "IvankaTrump"]

# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

plt.figure(figsize=(22,16))
nx.draw(G, 
        pos = pos, 
        node_size = 10,
        node_color = "mediumslateblue",
        edge_color = "lightgray",
        width = 0.5)

for i in trump_graph:
    nx.draw(G.subgraph(["realDonaldTrump", i]),
            pos = pos, 
            node_size = 70,
            node_color = "royalblue",
            edge_color = "dimgray",
            width = 0.5,
            with_labels = True)
nx.draw(G.subgraph("realDonaldTrump"), 
        pos = pos, 
        node_size = 500, 
        node_color = "royalblue",
        edge_color = "dimgray",
        width = 0.5,
        with_labels = True)

# plt.savefig("img/Trump_Friends.png")

Let's try to automate this procedure since creating a list for every account is obviously not feasible.

* The following two functions `get_friends_graph()` and `get_followers_graph()` return a list with the friends or followers of the account passed to the respective function. 
* The result of this function can be passed to the `G.subgraph` function inside a `nx.draw` function and it will create a subnetwork for the respective account.
* There are two options:
    1. Passing the list directly to the `G.subgraph` function will create a "complete" network of all accounts connected with the initial account and **connections in between them**.
    2. Looping over the list and iteratively drawing one point after another $\rightarrow$ this will plot the subnetwork of the friends or followers, **only** connected to the initial account.

In [ ]:
def get_friends_graph(account):
    graph = []
    cache = []
    for i in data:
        if account in i[0]:
            cache.append(i)
    
    graph.append(cache[1][0])
    
    for j in cache:
        graph.append(j[1])
        
    return graph

In [ ]:
def get_followers_graph(account):
    graph = []
    cache = []
    for i in data:
        if account in i[1]:
            cache.append(i)
            
    graph.append(account)
    
    for j in cache:
        graph.append(j[0])
    
    return graph

Now let's draw a fully automated plot of account `realDonaldTrump`. The first plot corresponds to option 1. above (creating a fully-connected network) while the second plot corresponds to option 2. and displays all the accounts that *follow* `realDonaldTrump`. The number of his friends is only 3 (the function returns 4 as his own account is included):

In [ ]:
len(get_friends_graph("realDonaldTrump"))

#### Option 1: Fully connected network

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

plt.figure(figsize=(20,14))
nx.draw(G, 
        pos = pos, 
        node_size = 10,
        node_color = "royalblue",
        edge_color = "lightgray",
        width = 0.5)

# Change Account Name to draw graph here (2 instances):
nx.draw(G.subgraph(get_followers_graph("realDonaldTrump")),
            pos = pos, 
            node_size = 60,
            node_color = "mediumblue",
            edge_color = "dimgray",
            width = 0.5,
            with_labels = True)

#### Option 2: Plot followers only

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

plt.figure(figsize=(22,16))
nx.draw(G, 
        pos = pos, 
        node_size = 10,
        node_color = "mediumslateblue",
        edge_color = "lightgray",
        width = 0.5)

# Change Account Name to draw graph here (2 instances):
for i in get_followers_graph("realDonaldTrump")[1:]:
    nx.draw(G.subgraph(["realDonaldTrump", i]),
            pos = pos, 
            node_size = 60,
            node_color = "royalblue",
            edge_color = "dimgray",
            width = 0.5,
            with_labels = True)

nx.draw(G.subgraph("realDonaldTrump"), 
        pos = pos, 
        node_size = 500, 
        node_color = "royalblue",
        edge_color = "dimgray",
        width = 0.5,
        with_labels = True)

# plt.savefig("img/Trump_Followers.png")

### Plot of the subnetwork with RC Coefficient and Node Degrees

First, we plot the network with the **RC coefficients as node labels**. Relabelling can easily be done with the `draw_networkx_labels` command. This command takes a dictionary which is mapped with the nodes, so the keys of the dict that is passed to this function have to be the same as the nodes in the network.

In [ ]:
rc_accounts_rounded = rc_accounts.copy()

for key, value in rc_accounts_rounded.items():
    if value < 1.0:
        v = round(value, 5)
        rc_accounts_rounded[key] = v

# print(rc_accounts_rounded)

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

# Define node sizes:
sizes = [v**3 + 70 for v in list(rc_accounts.values())]

plt.figure(figsize=(22,16))
nx.draw(G, 
        pos = pos, 
        node_size = sizes,
        node_color = "royalblue",
        edge_color = "lightgray",
        width = 0.5)

nx.draw_networkx_nodes(rc_1, 
        pos = pos, 
        node_size = 60, 
        node_color = "crimson")

nx.draw_networkx_labels(G, 
                        pos = pos, 
                        labels = rc_accounts_rounded,
                        verticalalignment = "bottom")

plt.savefig("img/RCC_1_No_Subnet.png")

Now, the same code with Account Names as Labels only for those where RCC = 1:

Now, we plot the entire network with the **node sizes proportionally to the node degrees**. First we create a list of the proportional sizes using a monotonic transformation:

Then, we create a dictionary (`degree_dict`) which is used for mapping the node degrees as node labels:

In [ ]:
degree_dict = {}

for i, j in G.degree():
    element = {i:j}
    degree_dict.update(element)

Finally, the plot:

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(G)

# Define node sizes:
sizes = [d[1]**1.7 + 70 for d in G.degree()]

plt.figure(figsize=(22,16))
nx.draw(G, 
        pos = pos, 
        node_size = sizes,
        node_color = "royalblue",
        edge_color = "lightgray",
        width = 0.5)
nx.draw_networkx_labels(G, 
                        pos = pos, 
                        labels = degree_dict)

plt.savefig("img/Node_Degree_Proportional.png")

### Plot of the subnetwork -- only Nodes with RCC = 1

First, create a graph with only the nodes with RCC = 1. We will use the `rc_1` list created above and map it to the entire `data` dataset.

In [ ]:
# Create list of tuples which will be used for creating the graph:
friend_cache = []
for d in data:
    if d[1] in rc_1:
        friend_cache.append(d)

rc_1_list = []
for f in friend_cache:
    if f[0] in rc_1:
        rc_1_list.append(f)

# Create networkx.Graph from rc_1_list:
RC1_G = nx.Graph()
RC1_G.add_nodes_from(r[0] for r in rc_1_list)
RC1_G.number_of_nodes()
RC1_G.add_edges_from(rc_1_list)

Create list of degrees:

In [ ]:
RC1_degrees = list(RC1_G.degree)
RC1_degrees
RC1_degrees

Create Plot:

In [ ]:
# For reproducibility of the plot:
np.random.seed(42)

# Define layout of the plot:
pos = nx.spring_layout(RC1_G)

size = list(d[1]**2 + 70 for d in RC1_G.degree())

plt.figure(figsize=(20,14))
nx.draw(RC1_G,
        pos = pos,
        node_size = size)

Calculate Rich Club Coefficient for this particular subnetwork:

In [ ]:
subnet_rcc = nx.rich_club_coefficient(RC1_G, normalized = False, seed = 42)
subnet_rcc

There are hardly any nodes within these network with a node degree < 23. Almost all nodes within this subnetwork are connected to **all other nodes within the subnetwork**. Further visualization beyond the one above wouln't make much sense since these are the only accounts that are not connecte to all other nodes:

* GOPChairwoman, Node Degree: 22
* DailyCaller, Node Degree: 22
* RepMattGaetz, Node Degree: 22
* ByronYork, Node Degree: 20

**This shows that there is a highly connected rich club present in the network**.